In [167]:
import pandas as pd 
import numpy as np
import tensorly as tl
import plotly.express as px
from tensorly.decomposition import CP,tucker, parafac

We apply differencing for the market capitalization because it is non-stationary. 
Prices are not normalized.

In [214]:
df_FR = pd.read_excel('FINANCIAL_RATIOS.xlsx', index_col =[0], skiprows=[1,2])
df_FR.drop(list(df_FR.filter(regex='TSCO')), axis=1, inplace = True)
df_FR.drop(list(df_FR.filter(regex='TTWO')), axis=1, inplace = True)
df_FR.drop(list(df_FR.filter(regex='AXTI')), axis=1, inplace = True)
df_FR.drop(list(df_FR.filter(regex='DISH')), axis=1, inplace = True)
df_FR.drop(list(df_FR.filter(regex='TTEC')), axis=1, inplace = True)
df_FR.drop(list(df_FR.filter(regex='ELTK')), axis=1, inplace = True)






FR_dic = {}
FR_list = ['PE', 'PX_LAST', 'PS','PB','MCAP']
n = 0 

for i,name in enumerate(FR_list) : 
    FR_dic[name] = (df_FR.iloc[:,n:n+215]
                    .set_index(df_FR.index)
                    # .drop(list(df_FR.filter(regex='TSCO')))
                    # .drop(df.filter(regex='TTWO').columns, axis=1)
                    # .drop(df.filter(regex='AXTI').columns, axis=1)
                    # .drop(df.filter(regex='DISH').columns, axis=1)
                    # .drop(df.filter(regex='TTEC').columns, axis=1)
                    # .drop(df.filter(regex='ELTK').columns, axis=1)
                    )
                    # .drop(list(df_FR.filter(regex='TSCO')))
                    # .drop(list(df_FR.filter(regex='TSCO')))

    if name not in ['PX_LAST', 'MCAP'] : 
        FR_dic[name] = (FR_dic[name] - FR_dic[name].mean()) / FR_dic[name].std()
        
    else :
        FR_dic[name] = FR_dic[name] - FR_dic[name].shift(+1)
    FR_dic[name] = FR_dic[name][1:]
    n +=217
COLUMN_NAMES = FR_dic['PE'].columns
















KeyError: "['TSCO US Equity.1' 'TSCO US Equity.2' 'TSCO US Equity.3'\n 'TSCO US Equity.4'] not found in axis"

In [209]:
test = FR_dic['PB']
test.loc[:, test.isna().any()]


# for key, val in FR_dic.items(): 
#     print(val.shape)
#     val.dropna(axis = 0, inplace = True)
#     print(val.shape)


,DISH US Equity.3,TTEC US Equity.3,ELTK US Equity.3
2006-11-17,NaN,NaN,NaN
2006-11-24,NaN,NaN,NaN
2006-12-01,NaN,NaN,NaN
2006-12-08,NaN,NaN,NaN
2006-12-15,NaN,NaN,NaN
...,...,...,...
2022-04-15,-0.471156,1.535977,-0.415678
2022-04-22,-0.472524,1.394399,-0.418571
2022-04-29,-0.473656,1.245173,-0.414738
2022-05-06,-0.477016,0.834680,-0.423768


In [160]:
FR_dic['PE']

,AMAT US Equity,COKE US Equity,WDFC US Equity,AAPL US Equity,KLAC US Equity,SEIC US Equity,CSPI US Equity,ALOT US Equity,AMGN US Equity,CAMP US Equity,...,ENTG US Equity,HBIO US Equity,HSTM US Equity,ILMN US Equity,ISRG US Equity,LPSN US Equity,MRVL US Equity,SLAB US Equity,TTMI US Equity,CCRN US Equity
2006-11-17,-0.177508,-0.119575,-0.915216,2.109720,0.331890,2.324358,1.127707,-0.389416,1.716510,-0.662353,...,-0.248954,-0.296971,-0.801469,-0.164106,-0.584526,-0.951002,-0.344092,-0.316539,-0.407508,-0.265605
2006-11-24,-0.174555,-0.122687,-0.912640,2.414422,0.333059,2.250332,1.112047,-0.366602,1.715521,-0.661792,...,-0.208069,-0.300016,-0.811713,-0.170129,-0.517713,-0.948835,-0.326946,-0.310508,-0.377562,-0.272915
2006-12-01,-0.180387,-0.142948,-0.809835,2.398076,0.291103,2.160493,1.227944,-0.355740,1.401284,-0.638370,...,-0.271846,-0.297624,-0.793789,-0.174522,-0.466832,-0.992176,-0.331453,-0.336937,-0.383462,-0.287757
2006-12-08,-0.174924,-0.105216,-0.763976,2.236764,0.307181,2.354706,1.159031,-0.352482,1.451653,-0.637006,...,-0.227695,-0.295775,-0.765624,-0.162033,-0.396957,-1.013845,-0.325847,-0.313047,-0.410684,-0.272029
2006-12-15,-0.177434,-0.105216,-0.717568,2.208302,0.279726,2.388694,1.071325,-0.366602,1.485892,-0.622501,...,-0.177005,-0.296210,-0.719535,-0.165982,-0.419430,-1.025763,-0.333541,-0.310904,-0.422932,-0.244559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-15,-0.184846,-0.341016,0.916766,0.810873,-0.320839,-1.397869,0.196164,-0.259704,-0.036205,1.936723,...,0.695321,-0.247252,0.909277,4.811261,0.766919,0.849110,0.891365,2.674924,-0.208848,-0.662176
2022-04-22,-0.185471,-0.351724,0.962499,0.742543,-0.308869,-1.443310,0.196164,-0.243138,-0.116377,1.936723,...,0.757423,-0.248293,0.843009,4.811261,0.443265,0.849110,0.891365,2.674924,-0.197315,-0.660694
2022-04-29,-0.188203,-0.371646,0.819392,0.661713,-0.330326,-1.512375,0.196164,-0.202649,-0.491283,1.936723,...,0.726368,-0.258704,0.801083,4.811261,0.289013,0.849110,0.891365,2.674924,-0.189628,-0.666067
2022-05-06,-0.185805,-0.346429,0.783287,0.654489,-0.278636,-1.526309,0.196164,-0.255100,-0.418992,1.936723,...,0.815401,-0.284035,0.799728,4.811261,0.118077,0.849110,0.891365,2.674924,-0.158333,-0.670792


In [17]:
df_TI = pd.read_excel('TECHNICAL_INDICATORS.xlsx',skiprows=[0])


In [72]:
TI_dic = {}
TI_list = ['ATR', 'BB', 'MACD_DIFF','MOM','ROC', 'RSI']
n = 1
for _, TI in enumerate(TI_list) : 
    filter_col = [col for col in df_TI if col.startswith(TI)]

    TI_dic[TI] = (df_TI[filter_col]
                 .set_axis(COLUMN_NAMES, axis=1)
                 .set_index(df_TI['Dates'])
                 .iloc[:-3]
                 )

In [81]:
df_PX = pd.read_excel('PRICE_DATA.xlsx', index_col =[0], skiprows=[0])

In [95]:
PX_dic = {}

PX_list = ['PX_HIGH', 'PX_LOW', 'PX_MID', 'PX_LAST', 'PX_VOL']


n = 0 

for i,name in enumerate(PX_list) : 
    PX_dic[name] = (df_PX.iloc[:,n:n+215]
                    .set_index(df_PX.index)
                    .iloc[:-2]
                    .iloc[1:]
                    )
    n +=215
    
    # FR_dic[name] = (FR_dic[name] - FR_dic[name].mean()) / FR_dic[name].std()

# COLUMN_NAMES = PX_dic['PE'].columns



PX_dic['PX_HIGH']

,PX_HIGH,PX_HIGH.1,PX_HIGH.2,PX_HIGH.3,PX_HIGH.4,PX_HIGH.5,PX_HIGH.6,PX_HIGH.7,PX_HIGH.8,PX_HIGH.9,...,PX_HIGH.205,PX_HIGH.206,PX_HIGH.207,PX_HIGH.208,PX_HIGH.209,PX_HIGH.210,PX_HIGH.211,PX_HIGH.212,PX_HIGH.213,PX_HIGH.214
Dates,,,,,,,,,,,,,,,,,,,,,
2006-11-10,17.9900,63.000,33.99,3.025,50.250,28.840,8.3500,10.24,75.84,7.10,...,10.06,3.9976,3.500,22.465,11.310,6.000,19.639,33.64,12.200,24.0740
2006-11-17,18.8700,65.450,33.08,3.082,52.090,29.745,8.6399,10.00,74.13,7.28,...,10.65,3.8988,3.840,20.620,10.787,5.640,20.160,34.85,12.750,22.4800
2006-11-24,18.5500,65.450,32.88,3.324,52.770,29.870,8.4800,9.91,73.48,7.29,...,10.87,3.7240,3.970,20.270,11.326,5.390,21.350,35.29,12.990,21.0000
2006-12-01,18.6100,64.540,32.93,3.327,52.130,29.400,8.8000,10.00,72.52,7.26,...,10.86,3.6860,3.980,19.670,11.620,5.250,20.950,35.08,12.860,20.4900
2006-12-08,18.7900,66.780,33.03,3.298,52.400,29.975,8.8000,10.10,70.45,7.37,...,10.91,3.7924,4.015,20.745,11.887,5.130,21.850,35.12,13.340,20.5800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-15,119.2600,503.500,195.30,171.270,345.135,59.330,7.4700,15.27,258.45,6.52,...,114.46,6.3100,20.320,360.155,287.150,26.660,63.750,138.92,13.920,22.0700
2022-04-22,122.3700,508.175,194.55,171.530,354.970,59.930,7.8311,13.59,258.39,6.09,...,122.32,6.1000,20.420,352.955,308.000,25.915,65.170,138.50,14.820,22.3600
2022-04-29,116.1600,499.960,194.76,166.200,342.265,57.880,7.7000,13.47,253.00,6.01,...,116.30,5.9000,20.680,321.190,259.720,24.140,62.730,141.49,14.410,20.5900


In [178]:
feature_tensor = np.stack(FR_dic.values(), axis=2)

# print(feature_tensor.shape)
# core,factors = tucker(feature_tensor, rank = [809,50,5])



[[[-1.77508391e-01  5.00000000e-01  8.75841793e-02 -1.79575203e-01
    6.89958000e+02]
  [-1.19575392e-01  4.19000000e+00 -3.11205911e-01  1.39138424e+00
    3.81432000e+01]
  [-9.15215680e-01 -3.80000000e-01 -8.90951729e-01 -8.89568763e-01
   -6.46830000e+00]
  ...
  [-3.16539309e-01  1.26000000e+00  4.68319795e-01  1.92715199e-01
    6.88071000e+01]
  [-4.07508152e-01  1.30000000e-01  5.24649776e+00  1.69987660e+00
    8.06290000e+00]
  [-2.65604762e-01 -1.34000000e+00  2.72558684e+00 -8.45527780e-02
   -4.28820000e+01]]

 [[-1.74554969e-01  4.00000000e-01  1.48766399e-01 -1.44010901e-01
    5.51966400e+02]
  [-1.22686855e-01 -3.90000000e-01 -3.26627696e-01  1.36073126e+00
   -3.55030000e+00]
  [-9.12639940e-01  5.00000000e-02 -8.89454435e-01 -8.88623277e-01
    8.51100000e-01]
  ...
  [-3.10507564e-01  7.60000000e-01  5.28310859e-01  2.86930263e-01
    4.15027000e+01]
  [-3.77562467e-01  6.60000000e-01  5.69375517e+00  1.98505858e+00
    2.77206000e+01]
  [-2.72915168e-01 -3.3000000

C:\Users\zackx\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):
